# Big Data: Project Phase 2

### By Tormod Alf Try Tufteland and Lars Henrik Berg-Jensen

#### Imports

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
import os, shutil, datetime
from collections import Counter
from operator import add, itemgetter

#### Creating Spark Context

In [ ]:
master = "local[4]"
appName = "phase2"
conf = SparkConf().setAppName(appName).setMaster(master)

In [ ]:
sc = SparkContext(conf=conf)

In [ ]:
sc.setLogLevel("WARN")

#### Creating RDD from geotweets.tsv

In [ ]:
rdd = sc.textFile("data/geotweets.tsv")

In [ ]:
sampled_rdd = rdd.sample(False, 0.1, 5)

#### Creating RDD arrays by splitting on tabs

In [ ]:
rdd_list = rdd.map(lambda x: x.split('\t'))

In [ ]:
sampled_rdd_list = sampled_rdd.map(lambda x: x.split('\t'))

#### Extracting place and tweet into new RDD

In [ ]:
# Using sampled or real RDD list
rdd_used = rdd_list
#rdd_used = sampled_rdd_list

In [ ]:
input_tweet = "#hiring"
input_tweet_list = input_tweet.split(' ')
input_tweet_rdd = sc.parallelize(input_tweet.split(' ')).map(lambda x: x.lower())

In [ ]:
total_tweet_count = rdd_used.count()
total_tweet_count

In [ ]:
place_and_tweet = rdd_used.map(lambda x: (x[4], x[10]))

In [ ]:
place_and_tweets = rdd_used.map(lambda x: (x[4], x[10]))\
                    .mapValues(lambda x: x.split(' '))\

In [ ]:
def findNumberOfTweetsWithWord(dataset, word):
    return dataset.filter(lambda x: word in x).count()

In [ ]:
#dataset: rdd of (place, tweets)
def findProbabilityOfPlace(dataset, place, tweet_rdd):
    tweets_from_place = dataset.filter(lambda x: x[0] == place)
    nr_tweets_place = tweets_from_place.count()
    tweets_from_place_list = tweets_from_place.map(lambda x: x[1].split(' '))
    #word_tweet_count = tweet_rdd.map(lambda x: (x, findNumberOfTweetsWithWord(tweets_from_place_list, x)))
    word_tweet_count = list(map(lambda x: (x, findNumberOfTweetsWithWord(tweets_from_place_list, x)), input_tweet_list))
    return word_tweet_count

In [ ]:
result_rdd = findProbabilityOfPlace(place_and_tweet, 'Belm, Par', input_tweet_rdd)

In [ ]:
def findPlaceOfTweet(dataset, tweet, output):
    number_of_tweets = dataset.count()
    
    #Split input tweet into list of words and put to lowercase 
    #For all tweets in dataset for each place, count the number of tweets where word occurs
    #For each place in dataset, calculate the probability using naive bayes
    #Sort the result by probability
    #Return the place with max probability
    pass

In [ ]:
tweets_by_city = place_and_tweets.map(lambda x: (x[0], list(map(str.lower, x[1]))))

In [ ]:
tweets_per_place = place_and_tweet.map(lambda x: (x[0], 1)).countByKey().items()
tweets_per_place = sc.parallelize(tweets_per_place).collectAsMap()

In [ ]:
def addIfWordInTweet(word, value, tweet_list):
    if word in tweet_list:
        value += 1
    return value
        
def to_dict(tweet_list):
    counter_dict = {k: 0 for k in input_tweet_list}
    #counter_dict = {k: func(tweet_list, k) for k in new_dict.items()}
    return counter_dict
    
def add(counter_dict, tweet_list):
    return {k: addIfWordInTweet(k, v, tweet_list) for k, v in counter_dict.items()}


def merge(dict1, dict2):
    new = {**dict1, **dict1}
    return new

counted_tweets_with_word_by_city = tweets_by_city.combineByKey(to_dict, add, merge)

In [ ]:
counted_tweets_with_word_by_city.collect()[0:50]

In [ ]:
#dataset: rdd of (place, {word1: count, word2:count ...})
def findProbabilityOfPlace(dataset, place):
    nr_tweets_place = tweets_per_place[place]
    probability = dataset.filter(lambda x: x == place)
    probability = (nr_tweets_place / total_tweet_count) * 
    return word_tweet_count

In [ ]:
from functools import reduce
def calculateProbability(place, data_dict):
    nr_tweets_place = tweets_per_place[place]
    initial_value = nr_tweets_place / total_tweet_count 
    probability = reduce(lambda x, value: x * value / nr_tweets_place, data_dict.values(), initial_value)
    return probability

In [ ]:
def findProbabilities(dataset):
    probabilities = dataset.map(lambda x: (x[0], calculateProbability(x[0], x[1])))
    return probabilities

In [ ]:
probabilities = findProbabilities(counted_tweets_with_word_by_city)

In [ ]:
sorted_probabilities = sorted(probabilities.collect(), key=lambda x: x[1], reverse=True)

In [ ]:
probabilities.max(key=lambda x: x[1])